In [1]:
import os, re, time
from dotenv import load_dotenv 
import json
from utils import *
from web3 import Web3
import pandas as pd
from pprint import pprint
from utils import *
from tqdm import tqdm
from decimal import Decimal
from datetime import datetime
import numpy as np
import mplfinance as mpf

# Get the config file
configObj = ConfigManager("config.json")
appInfo, configData = configObj.load_config()
nodeUrl = appInfo["alchemy_url"]+appInfo["alchemy_key"]


In [2]:

# Replace these values with your specific pair
PAIR_ADDRESS = "0x5c7f3aaba7943d90f10166ed53f9179ce7d8d989"; 
RPC_URL = nodeUrl
web3Obj = Web3(Web3.HTTPProvider(nodeUrl))

# Get transactions from the last 1000 blocks
end_block = 21400000 + 80000
from_block = 21400000 - 60000
end_block = web3Obj.eth.block_number
from_block = end_block - 40000

handler = ETH_Handler(web3Obj)
pairInfo = handler.get_pair_info(PAIR_ADDRESS)
quoteAssetIndex = 0 if pairInfo["token0"]["symbol"] == "WETH" else 1 if pairInfo["token1"]["symbol"] == "WETH" else None
quoteAsset = pairInfo[f"token{quoteAssetIndex}"]["symbol"]
baseAsset = pairInfo[f"token{quoteAssetIndex-1 if quoteAssetIndex == 1 else 1}"]["symbol"]
assetName = f"{baseAsset}/{quoteAsset}"

candleHandler = EVMTokenCandlestick("./resources/ETH_block_data", web3Obj, "ETH", True)
transactions = candleHandler.get_uniswap_pair_transactions(
    PAIR_ADDRESS, 
    from_block, 
    end_block, 
    None, 
    pairInfo["token0"]["decimals"], 
    pairInfo["token1"]["decimals"], 
    2 if pairInfo["version"]=="UNI_V2" else 3 if pairInfo["version"]=="UNI_V3" else -1,
    quoteAssetIndex,
    0.0001)
priceDf = candleHandler.process_transactions(transactions)
candleHandler.get_plot(priceDf, "4h", {"title":assetName})

Exception: Error fetching pair information: Not a uniswap V2 or V3 pair. are you sure what you have entered is for a conteract address?